In [1]:
# Initialise
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/beegfs/scratch_copy/ahringer/jj374/lab/relmapping


In [ ]:
# Load new annotation & test known corner cases
fp_regl = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv'
df_regl = pd.read_csv(fp_regl, sep='\t')
print('%d regions loaded' % (len(df_regl),))

def check_(igvstr, **kwargs):
    (chrom, start, end) = yp.parse_igvstr(igvstr)
    pos = int((start + end) / 2)
    df_ = df_regl.query('(chrom == "%s") & (start <= %d) & (%d < end)' % (chrom, pos, pos))
    assert len(df_) == 1
    for (k, v) in kwargs.items():
        print('\t'.join([igvstr, '%25s' % (k,), '%25s' % (v,), '%25s' % (df_[k].tolist()[0],)]))
        assert df_[k].tolist()[0] == v

#> rfc-4 (black element - promoter inside masked??)
#> zen-4 (black element - should be promoter?)sonRNA, I guess could mask promoter
#Rare but compelling cases of bidirectional promoters where one direction was protein-coding, and the other was an annotated non-coding RNA. Addressed by relaxing the non-coding RNA annotation. They're now treated more similarly to protein-coding annotations, so a site can be annotated as protein_coding on one strand, and snoRNA on the other strand. (Previously, any site that overlapped a non-coding RNA was excluded from any other annotation...)
#> attf-2 (black element maybe promoter?)
#Now annotated as a promoter for F09G2.2 (protein-coding gene, forward strabd), and F09G2.14 (snoRNA, reverse strand). Could also be a promoter for attf-2, but can't think of much one could do about this give current fragmented long cap data.
check_('chrIV:6117863-6118043',
       annot_fwd='coding_promoter', promoter_locus_id_fwd='zen-4',
       annot_rev='non-coding_RNA', promoter_locus_id_rev='M03D4.78')

## "Dual" coding/non-coding annotations: coding_promoter >> non-coding_RNA
check_('chrIII:6971981-6972101', 
       annot_fwd='coding_promoter', promoter_locus_id_fwd='rfc-4',
       annot_rev='coding_promoter', promoter_locus_id_rev='eef-1A.1') # alternative: mir-5549
check_('chrV:7196589-7196718',
       annot_fwd='coding_promoter', promoter_locus_id_fwd='F09G2.2',
       annot_rev='coding_promoter', promoter_locus_id_rev='attf-2') # alternative: F09G2.14

#> syx-4
#Compelling promoters; captured by relaxing the short cap threshold for calling a promoter that overlaps a first exon to having a single reproducible cap stack.
check_('chrII:13596635-13596738', annot_rev='coding_promoter', promoter_locus_id_rev='fbxc-23')
#check_('chrIV:8457013-8457189', annot_fwd='coding_promoter', promoter_locus_id_fwd='syx-4')

#> fbxc-15
#Fixed by tweaking the long cap jump test.
check_('chrII:979338-979455', annot_fwd='coding_promoter', promoter_locus_id_fwd='fbxc-15')

#> sri-5 (promoter annotated to rsr-1 only)
#> C25F9.6 (promoter annotate to C25F9.6 only) - looks like there is jump in txn?
#> F08F3.8 (promoter annotated to acl-6 only) - looks like there is longcap signal
#> rsd-2 (maybe one of the yellow elements promoter?) Looks like there is txn from furthest right yellow element
#Weak outron signal; now captured by requiring 1 long cap tag in each replicate, and at least 3 tags total (previously at least 2 tags in both replicates).
check_('chrI:11182502-11182624', annot_rev='coding_promoter', promoter_locus_id_rev='rsr-1')
check_('chrV:19405507-19405707', 
       annot_fwd='coding_promoter', promoter_locus_id_fwd='C25F9.10',
       annot_rev='coding_promoter', promoter_locus_id_rev='C25F9.6')
check_('chrV:5433170-5433353', annot_fwd='coding_promoter', promoter_locus_id_fwd='acl-6')

check_('chrIV:13541820-13541934', annot_rev='coding_promoter', promoter_locus_id_rev='rsd-2')

# Sites downstream within a long exon that should not be called promoters
check_('chrII:7150014-7150236', annot_fwd='no_transcription', annot_rev='transcription_initiation')
check_('chrI:2890703-2890828',  annot_fwd='unknown_promoter', associated_locus_id='Y71F9AL.6,arx-1')
check_('chrI:183422-183568', annot_fwd='transcription_initiation', associated_locus_id='atm-1')

# "Long distance" ncRNA annotations (should not happen)
check_('chrII:6891890-6892040', annot_fwd='unknown_promoter', annot_rev='transcription_initiation')

# Located between multiple alternative transcripts on the forward strand; looks like a true promoter
check_('chrII:6486433-6486590', 
        annot_fwd='coding_promoter', promoter_locus_id_fwd='T28D9.3', 
        annot_rev='coding_promoter', promoter_locus_id_rev='T28D9.9')

# addressed via closest_exon_flank ~< 250 (ncRNA downstream of site; exon1 from upstream gets mis-assigned)
check_('chrI:111191-111378', annot_fwd='non-coding_RNA', promoter_locus_id_fwd='F53G12.12')

# examples near/around first exons -- pass via allowing incr test

# Fwd strand promoter annotation is over two reverse strand gene bodies, not very compelling...
#check_p('chrI:5486294-5486413', 'coding_promoter', 'coding_promoter', 'rpl-19', 'C09D4.3') # allow raft approach for downstream?

# Non-coding RNAs within outrons should not 'block' upstream promoter annotation
check_('chrI:13324560-13324714', annot_fwd='coding_promoter', promoter_locus_id_fwd='gcy-35') # Reverse strand shows weak-gapped-but-compelling long cap?
check_('chrI:13372998-13373215', annot_rev='coding_promoter', promoter_locus_id_rev='sptf-3')
check_('chrIII:6971981-6972101', annot_fwd='coding_promoter', promoter_locus_id_fwd='rfc-4')
check_('chrIII:6971981-6972101', annot_rev='coding_promoter', promoter_locus_id_rev='eef-1A.1')
check_('chrI:13372998-13373215', annot_rev='coding_promoter', promoter_locus_id_rev='sptf-3')

# Long cap jump -- weak regions
check_('chrI:14316482-14316632', annot_fwd='pseudogene_promoter', promoter_locus_id_fwd='F49B2.4') # Also, reverse strand shows weak-gapped-but-compelling long cap (however no short cap!)?
check_('chrII:14437624-14437774', annot_fwd='unknown_promoter')
check_('chrI:5296989-5297097', annot_rev='coding_promoter', promoter_locus_id_rev='erm-1')

# Short cap 100..125bp from peak accessibility...
check_('chrI:6341909-6342059', annot_rev='unknown_promoter')
check_('chrII:14468955-14469118', annot_fwd='unknown_promoter')
# Short cap >125bp from peak accessibility...
#check_('chrII:14468955-14469118', annot_rev='unknown_promoter')

# Promoters initiating transcription within the UTR, downstream of the annotated TSS
check_('chrI:6011491-6011614', annot_rev='coding_promoter', promoter_locus_id_rev='rnf-1')
check_('chrI:7643236-7643386', annot_rev='coding_promoter', promoter_locus_id_rev='ncbp-2')
check_('chrV:1548208-1548358', annot_fwd='coding_promoter', promoter_locus_id_fwd='F52F10.2')
check_('chrV:12993480-12993612', annot_fwd='coding_promoter', promoter_locus_id_fwd='dsl-7')
check_('chrII:2320231-2320381', annot_fwd='coding_promoter', promoter_locus_id_fwd='ZK1240.3')
check_('chrI:5485932-5486082', annot_rev='coding_promoter', promoter_locus_id_rev='rpl-19')
check_('chrV:16262064-16262178', annot_fwd='coding_promoter', promoter_locus_id_fwd='nhr-127')
check_('chrII:6775038-6775173', annot_rev='coding_promoter', promoter_locus_id_rev='agr-1')
check_('chrII:2296175-2296310', annot_rev='coding_promoter', promoter_locus_id_rev='fbxc-25')

# Various weak-ish/corner cases where a clear unique promoter candidate exists
check_('chrI:5509301-5509451', annot_rev='coding_promoter', promoter_locus_id_rev='zipt-11') # upstream long cap overlaps upstream gene 3' end
check_('chrII:2772185-2772335', annot_rev='coding_promoter', promoter_locus_id_rev='sdz-10')
check_('chrV:5433187-5433337', annot_rev='coding_promoter', promoter_locus_id_rev='F08F3.8') # distal; site has no long cap (but has continuous txn)
check_('chrII:1818315-1818462', annot_rev='coding_promoter', promoter_locus_id_rev='fbxb-10') # short cap mode within first exon
#check_('chrII:13596635-13596738', annot_rev='coding_promoter', promoter_locus_id_rev='csc-1') # "distal"; no scap
check_('chrI:11182502-11182624', annot_fwd='coding_promoter', promoter_locus_id_fwd='sri-5') # no short cap
check_('chrI:11182502-11182624', annot_rev='coding_promoter', promoter_locus_id_rev='rsr-1')
check_('chrV:5433170-5433353', annot_rev='coding_promoter', promoter_locus_id_rev='F08F3.8') # no short cap
check_('chrI:13376619-13376731', annot_fwd='coding_promoter', promoter_locus_id_fwd='C01A2.9') # should be detected in d13_fwd -- padj 0.32, missed

# Very compelling jump in emb that's disregarded as short cap mode slightly further than 50bp
check_('chrII:1818315-1818462', annot_rev='coding_promoter', promoter_locus_id_rev='fbxb-10') # short cap within 1st exon
check_('chrII:1818315-1818462', annot_fwd='unknown_promoter')

# pseudogene promoters without short cap!
check_('chrII:14563893-14564084', annot_rev='pseudogene_promoter', promoter_locus_id_rev='C01G12.12')

# Additional weak cases
check_('chrV:706559-706684', annot_fwd='coding_promoter', promoter_locus_id_fwd='srbc-38')

In [ ]:
# Differences to previous github version
#fp_head_bed = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.bed_HEAD.bed'
#fp_head_tsv = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv_HEAD.tsv'
#!git show HEAD:{fp_regl} > {fp_head_tsv}
#!git show HEAD:annot/S2_regulatory_annotation/S2_regulatory_annotation.bed > {fp_head_bed}

# Diff against a specific earlier version
#fp_head_bed = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.bed_1030.bed'
#fp_head_tsv = 'annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv_1030.tsv'
#!git show 2040a6:annot/S2_regulatory_annotation/S2_regulatory_annotation.bed > {fp_head_bed}
#!git show 2040a6:annot/S2_regulatory_annotation/S2_regulatory_annotation.tsv > {fp_head_tsv}

fp_head_bed = 'annot/S2_regulatory_annotation/S2_regulatory_annotation_6Dec17.bed'
fp_head_tsv = 'annot/S2_regulatory_annotation/S2_regulatory_annotation_6Dec17.tsv'

df_head = pd.read_csv(fp_head_tsv, sep='\t')
m_diff_ = (df_head['annot_fwd'] != df_regl['annot_fwd']) | (df_head['annot_rev'] != df_regl['annot_rev'])# \
#| ((df_head['promoter_gene_id_fwd'] != df_regl['promoter_gene_id_fwd'])) \
#| ((df_head['promoter_gene_id_rev'] != df_regl['promoter_gene_id_rev'])) \
#| ((df_head['associated_gene_id'] != df_regl['associated_gene_id']))
print('%d regions annotated differently compared to last commit' % (sum(m_diff_)))

In [ ]:
# Randomly sample altered regions
df_diff = df_regl[['chrom', 'start', 'end']].copy()
df_diff['label_head'] = df_head['label']
df_diff['label_regl'] = df_regl['label']
df_diff['annot_rev_head'] = df_head['annot_rev']
df_diff['annot_rev_regl'] = df_regl['annot_rev']
df_diff['annot_fwd_head'] = df_head['annot_fwd']
df_diff['annot_fwd_regl'] = df_regl['annot_fwd']
yp.df_sample(df_diff.loc[m_diff_], 50)

In [ ]:
# Write a .bed-file of all the regions that changed
fp_diff = 'annot/S2_regulatory_annotation/S2_regulatory_annotation_diff.bed'
df_diff[m_diff_][['chrom', 'start', 'end']].to_csv(fp_diff, index=False, header=False, sep='\t')
!wc -l {fp_diff}

Possible additional improvements...

In [ ]:
# exon2 overlap rule is too conservative & masks even when txn clearly does not originate from exon
# => test for jump/incr at exon boundary & discard from annotation only if it tests positive
#check_annot_summary('chrIV:13588263-13588441', 'Y45F10B.12') trans-spliced promoter in L4?

In [ ]:
# extra long outrons???
# snt-2 has a distal promoter spanning 40kb and multiple genes (chrIII:787,433-829,218)
# => cut-off at e.g. 10kb?
#check_annot_summary('chrIII:828052-828187', '??')
#chrIII:4369629-4369711

In [ ]:

#check_annot_summary('chrII:11371902-11372117', '(C33B4.2)') # short cap 100bp downstream of annotated coding region
#check_annot_summary('chrI:5485952-5486062', 'rpl-19') # promoter assigned to un-translated tiny exon; scap not align
#check_annot_summary('chrII:11347905-11348076', 'B0491.6 / B0491.1') # long UTR; short cap precedes TSS
#check_annot_sumary('chrII:1818315-1818462', 'fbxb-10') # true promoter; scap ~60bp chrII:1821387-1821512
#check_annot_summary('chrIII:5876290-5876433', 'aldo-2')

In [ ]:
#> lir-2 (green element may be promoter?) Looks like a promoter, I think
#check_annot_summary('chrII:7667887-7668072', 'lir-2') # Tricky region, as short cap does not align with 5'